In [2]:
%pip install gymnasium
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import typing

# Videos
%pip install scikit-video
%pip install opencv-python
from IPython.display import Video
from skvideo.io import vwrite
import cv2
import numpy as np
import skvideo.io


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install stable-baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pygame
%pip install pymunk

import pygame
import pymunk
import pymunk.pygame_util


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
def reward_function(position: list) -> float:
  # Sample reward function for the ball_body_position
  if not (0 < position[0] < 600): # Penalty for not staying in bounds and falling off
    return -10
  if 550 < position[0] < 575: # Large reward for reaching right target
    return 100
  else:
    return -1 # Penalty for not reaching targets or staying in bounds

In [6]:
# Ball environment class
class BallEnv(gym.Env):

  #metadata
  metadata = {"render_modes": ["human"], "render_fps": 60}

  def __init__(self, reward_function: callable, max_time = 20):
    super(BallEnv, self).__init__()

    #NOTE: use np.arrays for larger spaces?

    # Stuff
    self.force = 50
    self.frame_count = 0
    self.done = False

    # Define discrete action space for 2 actions (moving left or moving right)
    self.action_space = gym.spaces.Discrete(2)

    # Define observation space (x, y directions)
    self.observation_space = gym.spaces.Box(low = 0,
                                            high = 600,
                                            shape = (2,),
                                            dtype = np.float32)

    # Define the actual game here with PyMunk, PyGame:
    # Initialize Pygame
    pygame.init()

    # Define time
    self.screen = pygame.display.set_mode((600, 400))
    self.clock = pygame.time.Clock()
    self.max_time = max_time
    self.elapsed_time = 0

    # Set up Pymunk space
    self.space = pymunk.Space()
    self.space.gravity = (0, 900)

    # Add a static floor
    self.floor_body = pymunk.Body(body_type=pymunk.Body.STATIC)
    self.floor_shape = pymunk.Segment(self.floor_body, (50, 300), (550, 300), 5)
    self.floor_shape.elasticity = 0.8
    self.space.add(self.floor_body, self.floor_shape)

    # Add a ball
    self.ball_body = pymunk.Body(1, pymunk.moment_for_circle(1, 0, 25))
    self.ball_body.position = (300, 275)
    self.observation = self.ball_body.position #Init observation?
    self.ball_shape = pymunk.Circle(self.ball_body, 25)
    self.ball_shape.elasticity = 0.8
    self.space.add(self.ball_body, self.ball_shape)

    # Draw options
    self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)

def _get_obs(self):
  return np.array([self.ball_body.position], dtype=np.float32)

def _get_info(self):
  return {
      "position": self.ball_body.position,
      "elapsed_time": self.elapsed_time
  }


def step(self, action):
  # Get the action
  self.ball_body.apply_force_at_local_point(-self.force) if action == 0 else self.ball_body.apply_force_at_local_point(self.force)

  # Step the simulation
  self.space.step(1 / 60.0)

  # Updating the elapsed time
  self.elapsed_time += self.clock.tick(60) #ms

  # Get the new state / observation
  # self.obs = self._get_obs()

  # Custom reward computation
  self.reward = self.reward_function(self.ball_body.position)

  # Check if the episode is done
  self.done = True if self.elapsed_time >= self.max_time*1000 or self.ball_body.position < 0 or self.ball_body.position > 600 else False

  self.observation = np.array([self.ball_body.position[0], self.ball_body.position[1]])

  info = {}

  return (self.observation, self.reward, self.done, info) #obs, reward, done, optional info

def reset(self, seed=None, options=None):
  # Reset self initial state, position, and velocity and time
  self.elapsed_time = 0
  self.ball_body.position = (300, 275)
  self.force = 50
  self.done = False
  self.floor_body = pymunk.Body(body_type=pymunk.Body.STATIC)
  self.floor_shape = pymunk.Segment(self.floor_body, (50, 300), (550, 300), 5)
  self.floor_shape.elasticity = 0.8

  # Add a ball
  self.ball_body = pymunk.Body(1, pymunk.moment_for_circle(1, 0, 25))
  self.ball_body.position = (300, 275)
  self.ball_shape = pymunk.Circle(self.ball_body, 25)
  self.ball_shape.elasticity = 0.8

  self.observation = np.array([self.ball_body.position[0], self.ball_body.position[1]])

  return self.observation

def render(self):
  self.scren.fill((0,0,0))
  self.space.debug_draw(self.draw_options)
  pygame.display.flip()
  self.clock.tick(60)

def close(self):
  pygame.quit()

def frame_update(self):
    self.screen.fill((0, 0, 0))
    self.space.debug_draw(self.draw_options)
    self.space.step(1 / 60.0)

    # Save current frame
    if frame_count > 400:
      return
    pygame.image.save(self.screen, f"frames/frame_{frame_count:04d}.png")
    frame_count += 1

    pygame.display.flip()

In [ ]:
env = BallEnv(reward_function = reward_function([300, 275]))

episodes = 50

for episode in range(episodes):
	done = False
	obs = env.reset()
	while True:#not done:
		random_action = env.action_space.sample()
		print("action",random_action)
		obs, reward, done, info = env.step(random_action)
		print('reward',reward)

action 0


NotImplementedError: 

: 

In [ ]:
!pass

In [ ]:
from stable_baselines3.common.env_checker import check_env

if __name__ == "__main__":
  env = BallEnv(reward_function = reward_function, max_time = 10)
  # print(env._get_obs())
  check_env(env)

# Create a PPO model with sb3
  model = PPO("MlpPolicy", env).learn(total_timesteps = 5000)

# Test the model
  obs, _ = env.reset()
  done = False
  while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
  env.close()

AssertionError: `reset()` must return a tuple (obs, info)

: 